### Imports
---

In [ ]:
import warnings
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import umap

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, minmax_scale
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.cluster import DBSCAN
from scipy.spatial.distance import directed_hausdorff
from scipy.spatial.transform import Rotation as R

warnings.simplefilter("ignore")

### Global Variable Definitions
---

In [ ]:
# The "root" folder that holds all the experiment data; must follow a strict folder structure.
ROOT = "data"
base_path = os.path.join(os.getcwd(), ROOT)

# The experiment type folder to look into, if each experiment has a "hand" and a "controller" folder.
FOLDER = "hand"

# Number of participants and experiments to index; TODO: Detect this automatically.
PARTICIPANTS = 4
EXPERIMENTS = 10

# The list of files to analyze.
FILE_LIST = [
        'Ball_1.csv', 'Ball2_1.csv', 'Ball3_1.csv', 'Ball4_1.csv', 'Ball5_1.csv',
        'Ball6_1.csv', 'Ball7_1.csv', 'Ball8_1.csv', 'Ball9_1.csv', 'Ball10_1.csv',
        'CubeMetal_1.csv', 'CubeNormal_1.csv', 'CubeWood_1.csv',
        'Cup_1.csv', 'Cup2_1.csv', 'Hammer_1.csv', 'Mallet_1.csv'
    ]

# The test size for the test/training data ratio. Standard is 80% training data, 20% test data.
TEST_SIZE = 0.6

IS_NORMALIZE = True

# Various features arrays categorized by the data they hold. 
imu_cols = [
    "imu_pos_x",
    "imu_pos_y", 
    "imu_pos_z", 
    "imu_rot_x", 
    "imu_rot_y", 
    "imu_rot_z"
    ]
object_cols = [
    # "lv_x", "lv_y", "lv_z", "av_x", "av_y", "av_z", 
    "pos_x", "pos_y", "pos_z"
    ]
hands_cols = [
    # 'lh_pos_x', "lh_pos_y", "lh_pos_z", "lh_rot_x", "lh_rot_y", "lh_rot_z",
    "rh_pos_x", "rh_pos_y", "rh_pos_z", "rh_rot_x", "rh_rot_y", "rh_rot_z"
]
fingers_cols = [
    # "lh_thumb_x", "lh_thumb_y", "lh_thumb_z",
    # "lh_index_x", "lh_index_y", "lh_index_z",
    # "lh_middle_x", "lh_middle_y", "lh_middle_z",
    # "lh_ring_x", "lh_ring_y", "lh_ring_z",
    # "lh_little_x", "lh_little_y", "lh_little_z",
    "rh_thumb_x", "rh_thumb_y", "rh_thumb_z",
    "rh_index_x", "rh_index_y", "rh_index_z",
    # "rh_middle_x", "rh_middle_y", "rh_middle_z",
    # "rh_ring_x", "rh_ring_y", "rh_ring_z",
    "rh_little_x", "rh_little_y", "rh_little_z"
]

# This is the important variable that will be used by the training model.
# Put any physical features you want to include here.
FEATURES_COLS = [object_cols, hands_cols, fingers_cols, imu_cols]
STAT_FEAT_LEN = 8

### Data Extraction
---

In [ ]:
def extract_features(df: pd.DataFrame):
    """
    This method extracts specific user defined features by processing raw data.\n
    The first section extracts statistical analysis of all columns indiscriminately.\n
    The rest is manual extraction of individual features from the raw data. 
    """
    features = []

    # Extract statistical features from all RAW DATA columns
    for col_group in FEATURES_COLS:
        group_data = df[col_group]
        # Calculated statistical features, modify as needed
        stats = [
            group_data.mean(),
            group_data.std(),
            group_data.max(),
            group_data.min(),
            group_data.quantile(0.25),
            group_data.quantile(0.75),
            group_data.skew(),
            group_data.kurtosis(),
        ]
        features.extend( [stat.values for stat in stats] )
        # features.extend( [extract_temporal_features(group_data)] )
    
    # Extract custom processed features.
    # Distance between the headset and the right hand.
    imu_pos_matrix = np.column_stack( [df["imu_pos_x"], df["imu_pos_y"], df["imu_pos_z"]] )
    rh_pos_matrix = np.column_stack( [df["rh_pos_x"], df["rh_pos_y"], df["rh_pos_z"]] )
    df["hand_head_distance"] =  np.linalg.norm( np.subtract( imu_pos_matrix, rh_pos_matrix) , axis=1)

    # Distance between index and thumb (right hand)
    rh_index_matrix = np.column_stack( [ df["rh_index_x"], df["rh_index_y"], df["rh_index_z"] ] )
    rh_thumb_matrix = np.column_stack( [ df["rh_thumb_x"], df["rh_thumb_y"], df["rh_thumb_z"] ] )
    df["thumb_index_distance"] =  np.linalg.norm( np.subtract(rh_index_matrix, rh_thumb_matrix) , axis=1)

    # Distance between index and middle (right hand)
    rh_middle_matrix = np.column_stack( [ df["rh_middle_x"], df["rh_middle_y"], df["rh_middle_z"] ] )
    df["middle_index_distance"] =  np.linalg.norm( np.subtract(rh_index_matrix, rh_middle_matrix) , axis=1)

    # Distance between ring and middle (right hand)
    rh_ring_matrix = np.column_stack( [ df["rh_ring_x"], df["rh_ring_y"], df["rh_ring_z"] ] )
    df["middle_ring_distance"] =  np.linalg.norm( np.subtract(rh_ring_matrix, rh_middle_matrix) , axis=1)

    # Distance between ring and little (right hand)
    rh_little_matrix = np.column_stack( [ df["rh_little_x"], df["rh_little_y"], df["rh_little_z"] ] )
    df["little_ring_distance"] =  np.linalg.norm( np.subtract(rh_ring_matrix, rh_little_matrix) , axis=1)

    # Distances between finger tips and hand
    df["rh_index_distance"] =  np.linalg.norm( np.subtract(rh_pos_matrix, rh_index_matrix) , axis=1)
    df["rh_thumb_distance"] =  np.linalg.norm( np.subtract(rh_pos_matrix, rh_thumb_matrix) , axis=1)
    df["rh_middle_distance"] =  np.linalg.norm( np.subtract(rh_pos_matrix, rh_middle_matrix) , axis=1)
    df["rh_ring_distance"] =  np.linalg.norm( np.subtract(rh_pos_matrix, rh_ring_matrix) , axis=1)
    df["rh_little_distance"] =  np.linalg.norm( np.subtract(rh_pos_matrix, rh_little_matrix) , axis=1)

    # Distance between pinky tip and object
    obj_pos_matrix = np.column_stack( [df["pos_x"], df["pos_y"], df["pos_z"]] )
    df["obj_pinky_distance"] =  np.linalg.norm( np.subtract( obj_pos_matrix, rh_little_matrix) , axis=1)
    
    complex_features = [
        # df["hand_head_distance"],
        # df["thumb_index_distance"],
        # df["middle_index_distance"],
        # df["middle_ring_distance"],
        # df["little_ring_distance"],
        # df["rh_index_distance"],
        # df["rh_middle_distance"],
        # df["rh_ring_distance"],
        # df["rh_little_distance"],
        # df["rh_thumb_distance"],
    ]

    for feature in complex_features:
        features.extend( [df_to_stats(feature)] )
    
    return np.concatenate(features)

def calculate_feature_importance(rfc: RandomForestClassifier | GradientBoostingClassifier):
    # Returns the feature importance list SPECIFICALLY for statistical features
    # Since each feature has ~8 statistical features, we sum them up and count them as one
    features = [item for sublist in FEATURES_COLS for item in sublist]
    feature_importance = pd.Series(rfc.feature_importances_)

    idx = 0
    result = {}

    for feature in features:
        total = sum(feature_importance[idx:idx+STAT_FEAT_LEN - 1])
        result[feature] = total
        idx += STAT_FEAT_LEN
    
    result = pd.Series(result)
    return result

def extract_temporal_features(df):
    features = []
    for col in df.columns:
        values = df[col].values

        # First-order difference (velocity changes)
        diff1 = np.diff(values, n=1)
        # Second-order difference (acceleration)
        diff2 = np.diff(values, n=2)
        # Second-order difference (jerk)
        diff3 = np.diff(values, n=3)

        # Autocorrelation (lag=1)
        autocorr = np.corrcoef(values[:-1], values[1:])[0, 1] if len(values) > 1 else 0

        # Store new features
        features.extend([
            np.mean(diff1), np.std(diff1), np.max(diff1), np.min(diff1),
            np.mean(diff2), np.std(diff2), np.max(diff2), np.min(diff2),
            np.mean(diff3), np.std(diff3), np.max(diff3), np.min(diff3),
            autocorr
        ])
    return np.array(features)

def extract_projection_features(df: pd.DataFrame):
    data = df
    # Step 1: Compute Relative Position
    relative_pos = data[['pos_x', 'pos_y', 'pos_z']].values - data[['imu_pos_x', 'imu_pos_y', 'imu_pos_z']].values

    def imu_to_rotation_matrix(imu_rot_x, imu_rot_y, imu_rot_z):
        return R.from_euler('xyz', [imu_rot_x, imu_rot_y, imu_rot_z], degrees=False).as_matrix()

    def compute_gaze_direction(rotation_matrix):
        return np.dot(rotation_matrix, np.array([1, 0, 0]))

    def project_trail(imu_pos, rotation_matrix, relative_pos):
        gaze_dir = compute_gaze_direction(rotation_matrix)
        proj_length = np.dot(relative_pos, gaze_dir)
        return imu_pos + gaze_dir * proj_length

    # Compute projected trail
    projected_trail = np.array([
        project_trail(data[['imu_pos_x', 'imu_pos_y', 'imu_pos_z']].iloc[i].values,
                      imu_to_rotation_matrix(*data[['imu_rot_x', 'imu_rot_y', 'imu_rot_z']].iloc[i].values),
                      relative_pos[i])
        for i in range(len(data))
    ])

    data[['trail_pos_x', 'trail_pos_y', 'trail_pos_z']] = projected_trail

    # Step 2: Compare Trajectory Shapes (Centroid & Curvature) 
    # dispersion is measured as the Euclidean distance between each point and the centroid (mean position) of the trajectory
    imu_centroid = data[['imu_pos_x', 'imu_pos_y', 'imu_pos_z']].mean().values
    obj_centroid = data[['pos_x', 'pos_y', 'pos_z']].mean().values
    trail_centroid = projected_trail.mean(axis=0)

    imu_disp = np.linalg.norm(data[['imu_pos_x', 'imu_pos_y', 'imu_pos_z']].values - imu_centroid, axis=1)
    obj_disp = np.linalg.norm(data[['pos_x', 'pos_y', 'pos_z']].values - obj_centroid, axis=1)
    trail_disp = np.linalg.norm(projected_trail - trail_centroid, axis=1)

    # Step 3: Distance Metrics (Hausdorff, measure of similarity between two sets of points) 
    hausdorff_obj_trail = directed_hausdorff(data[['pos_x', 'pos_y', 'pos_z']].values, projected_trail)[0]

    # Step 4: Statistical Analysis
    mean_diff_x, mean_diff_y, mean_diff_z = np.mean(np.abs(data[['pos_x', 'pos_y', 'pos_z']].values - projected_trail), axis=0)
    std_diff_x, std_diff_y, std_diff_z = np.std(data[['pos_x', 'pos_y', 'pos_z']].values - projected_trail, axis=0)
    corr_coeff = np.corrcoef(data[['pos_x', 'pos_y', 'pos_z']].values.flatten(), projected_trail.flatten())[0, 1]

    # Prepare data to be written to output
    feature_names = [
        "hausdorff_distance_object_vs_projected_trail", 
        "mean_diff_x", "mean_diff_y", "mean_diff_z", 
        "std_diff_x", "std_diff_y", "std_diff_z",
        "correlation_coefficient",
        "mean_imu_dispersion", "std_imu_dispersion",
        "mean_object_dispersion", "std_object_dispersion",
        "mean_projected_trail_dispersion", "std_projected_trail_dispersion"
    ]
    
    feature_values = [
        hausdorff_obj_trail, 
        mean_diff_x, mean_diff_y, mean_diff_z,
        std_diff_x, std_diff_y, std_diff_z,
        corr_coeff, 
        np.mean(imu_disp), np.std(imu_disp),
        np.mean(obj_disp), np.std(obj_disp),
        np.mean(trail_disp), np.std(trail_disp)
    ]

    features_df = pd.DataFrame()
    for idx, name in enumerate(feature_names):
        features_df[name] = feature_values[idx]
    
    print(features_df)
    return features_df

def df_to_stats(df: pd.DataFrame):
    return np.array([
        df.mean(),
        df.std(),
        df.max(),
        df.min(),
        df.quantile(0.25),
        df.quantile(0.75),
        df.skew(),
        df.kurtosis(),
    ])

def load_data(base_path):
    data = []
    labels = []
    successful_files = 0
    failed_files = 0
    missing_files = 0
    normalization_factors = {}

    for person in range(1, PARTICIPANTS+1):
        for folder in range(1, EXPERIMENTS+1):
            for file_name in FILE_LIST:
                file_path = os.path.join(base_path, str(person), FOLDER, str(folder), file_name)
                if os.path.exists(file_path):
                    df = read_csv_file(file_path)
                    if IS_NORMALIZE:
                        df, normalization_factors = normalize_features(df, "imu_pos_y", ["imu_rot_x"], user_id=str(person), normalization_factors=normalization_factors)
                    if df is not None and not df.empty:
                        try:
                            features = extract_features(df)
                            # features = extract_projection_features(df)
                            data.append(features)
                            labels.append(person - 1)
                            successful_files += 1
                            #print(f"Successfully processed: {file_path}")
                        except Exception as e:
                            failed_files += 1
                            print(f"Error processing {file_path}: {str(e)}")
                            print(e.with_traceback())
                else:
                    missing_files += 1
                    print(f"File does not exist: {file_path}")

    print(f"\nProcessing Summary:")
    print(f"Successful files: {successful_files}")
    print(f"Failed files: {failed_files}")
    print(f"Missing files: {missing_files}")
    print(f"Total files checked: {successful_files + failed_files + missing_files}")

    return np.array(data), np.array(labels)

def load_object_data_and_labels(base_path, object_name):
    object_data, object_labels = [], []
    normalization_factors = {}

    for person in range(1, PARTICIPANTS+1):
        for folder in range(1, EXPERIMENTS+1):
            file_path = os.path.join(base_path, str(person), FOLDER, str(folder), f'{object_name}_1.csv')
            if os.path.exists(file_path):
                try:
                    df = read_csv_file(file_path)
                    if IS_NORMALIZE:
                        df, normalization_factors = normalize_features(df, "imu_pos_y", ["imu_rot_x"], user_id=str(person), normalization_factors=normalization_factors)
                    if df is not None and not df.empty:
                        features = extract_features(df)
                        # features = extract_projection_features(df)
                        object_data.append(features)
                        object_labels.append(person - 1)
                except Exception as e:
                    continue
    return object_data, object_labels

def read_csv_file(file_path):
    try:
        with open(file_path, 'r') as file:
            header_line = file.readline().strip()
            columns = [col.strip() for col in header_line.split(',')]
        data = pd.read_csv(file_path, skiprows=1, header=None, names=columns)
        return data
    except Exception as e:
        print(f"Error reading {file_path}: {str(e)}")
        return None

def check_data_availability(base_path):
    print("\nChecking data availability:")
    for person in range(1, 5):
        print(f"\nPerson {person}:")
        for folder in range(1, 11):
            print(f"\nFolder {folder}:")
            for file_name in FILE_LIST:
                file_path = os.path.join(base_path, str(person), FOLDER, str(folder), file_name)
                status = "✓" if os.path.exists(file_path) else "✗"
                print(f"{status} {file_name}")

def normalize_features(df, reference_feature, other_features=[], user_id = None, normalization_factors={}):
    if user_id not in normalization_factors:
        # Compute initial value & scale factor from the reference feature
        initial_value = df[reference_feature].iloc[0]
        max_value = df[reference_feature].abs().max()

        # Avoid division by zero
        scale_factor = max_value if max_value != 0 else 1
        normalization_factors[user_id] = (initial_value, scale_factor)
    
    # Apply stored normalization factors
    initial_value, scale_factor = normalization_factors[user_id]
    df[reference_feature] = (df[reference_feature] - initial_value) / scale_factor

    # Apply the same transformation to other features
    for feature in other_features:
        df[feature] = (df[feature] - df[feature].iloc[0]) / scale_factor
    
    return df, normalization_factors

### Visualization Functions
---

In [ ]:
def visualize_tsne_by_object():
    plt.figure(figsize=(25, 15))
    n_rows = 3
    n_cols = 6
    object_types = set()
    for file in FILE_LIST:
        if file.endswith('_1.csv'):
            object_name = file.replace('_1.csv', '')
            object_types.add(object_name)
    plt.subplots_adjust(hspace=0.4, wspace=0.4)
    colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']

    for idx, object_name in enumerate(sorted(object_types)):
        object_data, object_labels = load_object_data_and_labels(base_path, object_name)

        if object_data:
            ax = plt.subplot(n_rows, n_cols, idx + 1)
            X_obj = np.array(object_data)
            y_obj = np.array(object_labels)
            # scaler = StandardScaler()
            scaler = MinMaxScaler(feature_range=(0, 1))
            X_scaled = scaler.fit_transform(X_obj)
            tsne = TSNE(n_components=2, random_state=42, perplexity=min(30, len(X_scaled)-1))
            X_tsne = tsne.fit_transform(X_scaled)
            X_tsne = scaler.fit_transform(X_tsne)
            for user in range(4):
                mask = y_obj == user
                plt.scatter(X_tsne[mask, 0], X_tsne[mask, 1],
                            c=[colors[user]], label=f'User {user+1}',
                            s=80, alpha=0.7, edgecolor='white', linewidth=0.5)

            plt.title(object_name, fontsize=12, pad=10, fontweight='bold')
            plt.xlabel('t-SNE 1', fontsize=10)
            plt.ylabel('t-SNE 2', fontsize=10)
            plt.xticks(fontsize=8)
            plt.yticks(fontsize=8)
            if idx == 0:
                plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=10, frameon=True)
            else:
                plt.legend([], [], frameon=False)
            plt.grid(True, linestyle='--', alpha=0.3)
            ax.spines['top'].set_visible(False)
            ax.spines['right'].set_visible(False)

    plt.tight_layout()
    plt.show()

def visualize_pca_by_object():
    plt.figure(figsize=(25, 15))
    n_rows = 3
    n_cols = 6
    object_types = set()
    for file in FILE_LIST:
        if file.endswith('_1.csv'):
            object_name = file.replace('_1.csv', '')
            object_types.add(object_name)
    plt.subplots_adjust(hspace=0.4, wspace=0.4)
    colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']

    for idx, object_name in enumerate(sorted(object_types)):
        object_data, object_labels = load_object_data_and_labels(base_path, object_name)

        if object_data:
            ax = plt.subplot(n_rows, n_cols, idx + 1)
            X_obj = np.array(object_data)
            y_obj = np.array(object_labels)
            # scaler = StandardScaler()
            scaler = MinMaxScaler(feature_range=(0,1))
            X_scaled = scaler.fit_transform(X_obj)
            pca = PCA(n_components=2, random_state=42)
            X_pca = pca.fit_transform(X_scaled)
            X_pca = scaler.fit_transform(X_pca)
            for user in range(4):
                mask = y_obj == user
                plt.scatter(X_pca[mask, 0], X_pca[mask, 1],
                            c=[colors[user]], label=f'User {user+1}',
                            s=80, alpha=0.7, edgecolor='white', linewidth=0.5)

            plt.title(object_name, fontsize=12, pad=10, fontweight='bold')
            plt.xlabel('PCA 1', fontsize=10)
            plt.ylabel('PCA 2', fontsize=10)
            plt.xticks(fontsize=8)
            plt.yticks(fontsize=8)
            if idx == 0:
                plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=10, frameon=True)
            else:
                plt.legend([], [], frameon=False)
            plt.grid(True, linestyle='--', alpha=0.3)
            ax.spines['top'].set_visible(False)
            ax.spines['right'].set_visible(False)

    plt.tight_layout()
    plt.show()

def visualize_umap_by_object():
    plt.figure(figsize=(25, 15))
    n_rows, n_cols = 3, 6
    object_types = {file.replace('_1.csv', '') for file in FILE_LIST if file.endswith('_1.csv')}

    plt.subplots_adjust(hspace=0.4, wspace=0.4)
    colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']

    for idx, object_name in enumerate(sorted(object_types)):
        object_data, object_labels = load_object_data_and_labels(base_path, object_name)

        if object_data:
            ax = plt.subplot(n_rows, n_cols, idx + 1)
            X_obj = np.array(object_data)
            y_obj = np.array(object_labels)

            # 🔹 Ensure Data Consistency Before UMAP
            if X_obj.shape[0] != len(y_obj):
                print(f"Skipping {object_name}: Feature and label size mismatch.")
                continue

            # 🔹 Standardize Data
            # scaler = StandardScaler()
            scaler = MinMaxScaler(feature_range=(0, 1))
            X_scaled = scaler.fit_transform(X_obj)

            # 🔹 Apply UMAP
            umap_model = umap.UMAP(n_components=2, n_neighbors=5, min_dist=0.01, random_state=42, metric="correlation")
            X_umap = umap_model.fit_transform(X_scaled)
            X_umap = scaler.fit_transform(X_umap)

            # db = DBSCAN()
            # X_dbscan_labels = db.fit_predict(X_umap)

            # 🔹 Verify UMAP Output Matches Labels
            if X_umap.shape[0] != len(y_obj):
                print(f"Skipping {object_name}: UMAP output and labels size mismatch.")
                continue  # Skip this object if still mismatched

            # 🔹 Plot Results
            for user in np.unique(y_obj):  # Dynamically get unique users
                mask = (y_obj == user)

                plt.scatter(
                    X_umap[mask, 0], X_umap[mask, 1],
                    c=[colors[user % len(colors)]], label=f'User {user+1}',
                    s=80, alpha=0.7, edgecolor='white', linewidth=0.5
                )

            plt.title(object_name, fontsize=12, pad=10, fontweight='bold')
            plt.xlabel('UMAP 1', fontsize=10)
            plt.ylabel('UMAP 2', fontsize=10)
            plt.xticks(fontsize=8)
            plt.yticks(fontsize=8)

            if idx == 0:
                plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=10, frameon=True)
            else:
                plt.legend([], [], frameon=False)

            plt.grid(True, linestyle='--', alpha=0.3)
            ax.spines['top'].set_visible(False)
            ax.spines['right'].set_visible(False)

    plt.tight_layout()
    plt.show()

### Training and Classification
---

In [ ]:
def train_and_evaluate_model(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE, random_state=42)
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
    rf_classifier.fit(X_train_scaled, y_train)
    y_pred = rf_classifier.predict(X_test_scaled)

    print("\nClassification Report:")
    print(classification_report(y_test, y_pred))

    plt.figure(figsize=(8, 6))
    sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='d', cmap='Blues')
    plt.title('Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.show()

    return rf_classifier, scaler

def print_classification_results(X, y, method="rfc", object_name=None):
    X_train, X_test, y_train, y_test = train_test_split(
        X, y,
        test_size=TEST_SIZE,
        random_state=42,
        stratify=y
    )

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    if method == "gbc":
        print("Using Gradient Boosting Classifier")
        rf_classifier = GradientBoostingClassifier(
            n_estimators=100, 
            random_state=42, 
            max_depth=10, 
            min_samples_split=5, 
            min_samples_leaf=2, 
            max_features="sqrt",
            
        )
    else:
        print("Using Random Forest Classifier")
        rf_classifier = RandomForestClassifier(
            n_estimators=100, 
            random_state=42,
            max_depth=10,
            min_samples_split=5,
            min_samples_leaf=2,
            max_features="sqrt",
            class_weight="balanced",
            n_jobs=-1
        )

    rf_classifier.fit(X_train_scaled, y_train)
    y_pred = rf_classifier.predict(X_test_scaled)
    feature_importance = calculate_feature_importance(rf_classifier)

    print("\nData Distribution:")
    unique, counts = np.unique(y, return_counts=True)
    print("Total samples per user:", dict(zip([f"User {i+1}" for i in unique], counts)))

    unique_test, counts_test = np.unique(y_test, return_counts=True)
    print("Test set samples per user:", dict(zip([f"User {i+1}" for i in unique_test], counts_test)))

    title = f"\nClassification Results for {object_name}" if object_name else "\nOverall Classification Results"
    print(title)
    print("-" * len(title))

    print("\nConfusion Matrix:")
    cm = confusion_matrix(y_test, y_pred)
    print("True\\Pred |  User1  User2  User3  User4")
    print("-" * 40)
    for i, row in enumerate(cm):
        print(f"User {i+1}    | {str(row).center(20)}")

    print("\nClassification Report:")
    print(classification_report(y_test, y_pred))

    print("\nTop 10 Features:")
    print(feature_importance.sort_values(ascending=False).head(10))
    print("\n")
    
    return rf_classifier

def print_perobject_classification_results():
    object_types = set()
    for file in FILE_LIST:
        if file.endswith('_1.csv'):
            object_name = file.replace('_1.csv', '')
            object_types.add(object_name)

    print("\nPer-Object Classification Results:")
    print("==================================")
    for object_name in sorted(object_types):
        object_data, object_labels = load_object_data_and_labels(base_path, object_name)

        if object_data:
            X_obj = np.array(object_data)
            y_obj = np.array(object_labels)
            print_classification_results(X_obj, y_obj, object_name=object_name)

### Main
---

In [ ]:
print(f"Loading data from: {base_path}")
X, y = load_data(base_path)

In [ ]:
def main():
    print_classification_results(X, y)
    print_classification_results(X, y, method="gbc")

    # Per-object classification results
    # print_perobject_classification_results()
    
    # Visualize clusters using the three methods
    
    # print("\nVisualizing clusters using t-SNE:")
    # visualize_tsne_by_object(X, y, FILE_LIST, base_path)

    # print("\nVisualizing clusters using PCA:")
    # visualize_pca_by_object(X, y, FILE_LIST, base_path)

    # print("\nVisualizing clusters using UMAP:")
    # visualize_umap_by_object(FILE_LIST, base_path)


In [ ]:
main()

In [ ]:
base_path = os.path.join(os.getcwd(), ROOT)
visualize_umap_by_object(FILE_LIST, base_path)